In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import re
import preprocessor as p
import emoji
import codecs
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = set(stopwords.words('english')) - set(['no', 'not'])

import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
tf.get_logger().setLevel('ERROR')

In [14]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

In [2]:
def write_files(dataframe):
    print("Writing files...")
    for i in tqdm(range(len(dataframe.loc[:, "Text"].values))):
        if i < len(dataframe.loc[:, "Text"].values)*0.66:
            if dataframe.loc[i, "Useful"] == True:
                file_name = str(i) + ".txt"
                with open('useful/train/true/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            else:
                file_name = str(i) + ".txt"
                with open('useful/train/false/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
        else:
            if dataframe.loc[i, "Useful"] == True:
                file_name = str(i) + ".txt"
                with open('useful/test/true/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])
            else:
                file_name = str(i) + ".txt"
                with open('useful/test/false/'+file_name, 'w', encoding='utf-8') as f:
                    f.write(dataframe.loc[i, "Text"])

In [3]:
def train_bert_model():
    AUTOTUNE = tf.data.AUTOTUNE
    batch_size = 32
    seed = 42

    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'useful/train',
        batch_size=batch_size,
        validation_split=0.2,
        subset='training',
        seed=seed)

    class_names = raw_train_ds.class_names
    train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

    val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'useful/train',
        batch_size=batch_size,
        validation_split=0.2,
        subset='validation',
        seed=seed)

    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        'useful/test',
        batch_size=batch_size)

    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

    bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
    tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
    tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    classifier_model = tf.keras.Model(text_input, net)

    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()
    epochs = 1
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)
    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')

    classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    print(f'Training model with {tfhub_handle_encoder}')
    history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

    loss, accuracy = classifier_model.evaluate(test_ds)
    return loss, accuracy

In [4]:
dataset = pd.read_csv('useful.csv')

In [5]:
write_files(dataset)

  2%|▏         | 332/15000 [00:00<00:04, 3318.47it/s]

Writing files...


100%|██████████| 15000/15000 [00:09<00:00, 1536.04it/s]


In [6]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 114s 710ms/step - loss: 0.0721 - binary_accuracy: 0.9778


In [7]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.lower()
    dataset.loc[i, "Text"] = v

In [8]:
write_files(dataset)

  4%|▎         | 557/15000 [00:00<00:02, 5542.82it/s]

Writing files...


100%|██████████| 15000/15000 [00:03<00:00, 4039.86it/s]


In [9]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 114s 712ms/step - loss: 0.0798 - binary_accuracy: 0.9741


In [10]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace('#', ' ')
    v = v.replace('@', ' ')
    v = p.clean(v)
    dataset.loc[i, "Text"] = v

In [11]:
write_files(dataset)

  2%|▏         | 307/15000 [00:00<00:04, 3063.17it/s]

Writing files...


100%|██████████| 15000/15000 [00:08<00:00, 1729.62it/s]


In [12]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 115s 717ms/step - loss: 0.0916 - binary_accuracy: 0.9704


In [15]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = v.replace("’", "'")
    for key in contractions:
        if key in v:
            v = re.sub(key, contractions[key], v)
    dataset.loc[i, "Text"] = v

In [16]:
write_files(dataset)

  2%|▏         | 367/15000 [00:00<00:03, 3669.12it/s]

Writing files...


100%|██████████| 15000/15000 [00:08<00:00, 1755.24it/s]


In [17]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 110s 690ms/step - loss: 0.0735 - binary_accuracy: 0.9798


In [18]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = codecs.decode(v, 'unicode_escape')
    v = emoji.demojize(v.encode('utf-16', 'surrogatepass').decode('utf-16'))
    dataset.loc[i, "Text"] = v

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: invalid escape sequence '\_'
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
write_files(dataset)

  1%|▏         | 221/15000 [00:00<00:06, 2124.47it/s]

Writing files...


100%|██████████| 15000/15000 [00:06<00:00, 2292.40it/s]


In [20]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 116s 723ms/step - loss: 0.0708 - binary_accuracy: 0.9790


In [21]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    v = re.sub("[^a-z0-9 ]", " ", v).strip()
    v = re.sub(" +", " ", v)
    dataset.loc[i, "Text"] = v

In [22]:
write_files(dataset)

  1%|▏         | 200/15000 [00:00<00:07, 1870.77it/s]

Writing files...


100%|██████████| 15000/15000 [00:06<00:00, 2204.18it/s]


In [23]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 113s 708ms/step - loss: 0.1197 - binary_accuracy: 0.9594


In [24]:
dataset = pd.read_csv('useful.csv')
for i, v in enumerate(dataset.loc[:, "Text"].values):
    temp_v = v.split()
    l = WordNetLemmatizer()
    v = [l.lemmatize(word) for word in temp_v if not word in stopwords]
    v = ' '.join(v)
    dataset.loc[i, "Text"] = v

In [25]:
write_files(dataset)

  3%|▎         | 398/15000 [00:00<00:03, 3975.10it/s]

Writing files...


100%|██████████| 15000/15000 [00:09<00:00, 1563.80it/s]


In [26]:
loss, accuracy = train_bert_model()

Found 9900 files belonging to 2 classes.
Using 7920 files for training.
Found 9900 files belonging to 2 classes.
Using 1980 files for validation.
Found 5100 files belonging to 2 classes.
Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
160/160 [==============================] - 113s 704ms/step - loss: 0.0740 - binary_accuracy: 0.9759
